# Importing  necessary liberaries

In [1]:
# !pip install pyspark

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,year, month, dayofmonth, date_format,hour,to_timestamp
from pyspark.sql.types import *
from google.colab import drive
import zipfile
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression,RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
import time


In [3]:
# creating the spark session
spark=SparkSession.builder.appName("NYC Taxi Fare Predction").getOrCreate()
spark

In [4]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Data Loading and Inspection

In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# variables
zip_path = "/content/drive/MyDrive/NYC_taxi_data/new-york-city-taxi-fare-prediction.zip"
extract_path = "/content/drive/MyDrive/NYC_taxi_data/"
train_file_path = "/content/drive/MyDrive/NYC_taxi_data/train.csv"
test_file_path = "/content/drive/MyDrive/NYC_taxi_data/test.csv"
number_of_rows = 200000
min_latitude, max_latitude = -90, 90
min_longitude, max_longitude = -180, 180

In [7]:
# function to extract zip file
def unzip_file(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
        print("File unzipped successfully")

# unzip_file(zip_path, extract_path)

In [8]:
!ls /content/drive/MyDrive/NYC_taxi_data/

GCP-Coupons-Instructions.rtf  new-york-city-taxi-fare-prediction.zip  test.csv
lr_model		      sample_submission.csv		      train.csv


In [9]:
# read csv file from google drive and show 10 rows from the dataframe
start_time = time.time()
df = spark.read.csv(train_file_path, header=True, inferSchema=True).limit(number_of_rows)
end_time = time.time()
print(f"Time taken to load the data: {end_time - start_time} seconds")

Time taken to load the data: 199.40736651420593 seconds


In [10]:
df_new = df.withColumn("pickup_datetime", to_timestamp(col("pickup_datetime"), "yyyy-MM-dd HH:mm:ss"))

In [11]:
df_new.show(10)

+--------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|                 key|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+--------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|2009-06-15 17:26:...|        4.5|2009-06-15 17:26:21|      -73.844311|      40.721319|        -73.84161|       40.712278|              1|
|2010-01-05 16:52:...|       16.9|2010-01-05 16:52:16|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|2011-08-18 00:35:...|        5.7|2011-08-18 00:35:00|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
|2012-04-21 04:30:...|        7.7|2012-04-21 04:30:42|       -73.98713|      40.733143|       -73.991567|       40.758092|              1|
|2010-03-09 07:51:...|     

In [12]:
# check for the columns, data types and if there are any null values
df_new.printSchema()

root
 |-- key: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)



In [13]:
# check for the number of partitons
num_partitions = df_new.rdd.getNumPartitions()
print("Number of partitions:", num_partitions)

Number of partitions: 1


In [14]:
# repartitioning the dataframe
df_new = df_new.repartition(10)

In [15]:
# count number of rows
row_count = df_new.count()
print("Number of rows:", row_count)

Number of rows: 200000


In [16]:
# checking for the statistical summery
summary_df =df_new.describe()
summary_df.show()

+-------+--------------------+------------------+------------------+------------------+------------------+-----------------+----------------+
|summary|                 key|       fare_amount|  pickup_longitude|   pickup_latitude| dropoff_longitude| dropoff_latitude| passenger_count|
+-------+--------------------+------------------+------------------+------------------+------------------+-----------------+----------------+
|  count|              200000|            200000|            200000|            200000|            199999|           199999|          200000|
|   mean|                NULL|11.342876949999926|-72.50612144955186|39.922325777255686|-72.51867346221104|39.92557945791378|        1.682445|
| stddev|                NULL|  9.83785478733004|11.608096802996164|10.048946659938098|10.724225862534947|6.751120031010629|1.30672964292037|
|    min|2009-01-01 00:35:...|             -44.9|           -736.55|      -3116.285383|       -1251.19589|      -1189.61544|               0|
|    m

**The minimum** fare is in negative, which is clearly erroneous (fares should not be negative).
**The maximum** fare is $93,963.36, which is extremely high and likely an outlier.

# Data Cleaning

In [17]:
# drop the rows with the null columns
df_new=df_new.na.drop()

In [18]:
# drop duplicates
df_new=df_new.dropDuplicates()

In [19]:
# filtering out the erroneous data
df_1 = df_new.filter(col('fare_amount') >= 0) \
               .filter((col('passenger_count') > 0) & (col('passenger_count') < 7)) \
               .filter((col('pickup_longitude') >= min_latitude) & (col('pickup_longitude') <= max_latitude)) \
               .filter((col('pickup_latitude') >= min_longitude) & (col('pickup_latitude') <= max_longitude)) \
               .filter((col('dropoff_longitude') >= min_latitude) & (col('dropoff_longitude') <= max_latitude)) \
               .filter((col('dropoff_latitude') >= min_longitude) & (col('dropoff_latitude') <= max_longitude))

# alternatively, df_new = df_new.where(col('fare_amount') >= 0)

In [20]:
# removeing outlier
q1 = df_1.approxQuantile("fare_amount", [0.25], 0.01)[0]
q3 = df_1.approxQuantile("fare_amount", [0.75], 0.01)[0]
iqr = q3 - q1

# defining upper and the lower bound
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

In [21]:
df_1 = df_1.filter(col('fare_amount') >= lower_bound) \
               .filter(col('fare_amount') <= upper_bound )

In [22]:
# extracting new columns from pickup_datetime
df_1 = df_1.withColumn("year", year(col("pickup_datetime"))) \
       .withColumn("month", month(col("pickup_datetime"))) \
       .withColumn("day", dayofmonth(col("pickup_datetime"))) \
       .withColumn("day_of_week", date_format(col("pickup_datetime"), 'u').cast(IntegerType())) \
       .withColumn("hour_of_day", hour(col("pickup_datetime"))) \
       .drop("key")

df_1.printSchema()

root
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- hour_of_day: integer (nullable = true)



In [23]:
df_1.show(10)

+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+----+-----+---+-----------+-----------+
|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|year|month|day|day_of_week|hour_of_day|
+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+----+-----+---+-----------+-----------+
|       12.9|2011-08-24 16:14:22|      -73.996097|      40.738571|       -73.955683|       40.765484|              1|2011|    8| 24|          3|         16|
|        4.5|2013-02-26 13:26:58|      -73.976059|      40.751548|       -73.975746|       40.744796|              1|2013|    2| 26|          2|         13|
|       10.5|2014-06-29 07:27:00|      -73.994757|        40.7183|       -73.972327|       40.746397|              1|2014|    6| 29|          7|          7|
|        6.5|2011-12-09 13:29:00|      -74.003652|      40

# Exploratory Data Analysis (EDA)

In [24]:
# create tempurarty view to execute sql queries
df_1.createOrReplaceTempView("NYC_taxi_data")

In [25]:
# Mean, median, and standard deviation for numerical columns
summery_stats = """
SELECT
    AVG(fare_amount) AS avg_fare_amount,
    PERCENTILE_APPROX(fare_amount, 0.5) AS median_fare_amount,
    STDDEV(fare_amount) AS stddev_fare_amount,
    AVG(pickup_longitude) AS avg_pickup_longitude,
    AVG(pickup_latitude) AS avg_pickup_latitude,
    AVG(dropoff_longitude) AS avg_dropoff_longitude,
    AVG(dropoff_latitude) AS avg_dropoff_latitude,
    AVG(passenger_count) AS avg_passenger_count
FROM NYC_taxi_data
"""
summary_stats = spark.sql(summery_stats)
summary_stats.show()

+----------------+------------------+------------------+--------------------+-------------------+---------------------+--------------------+-------------------+
| avg_fare_amount|median_fare_amount|stddev_fare_amount|avg_pickup_longitude|avg_pickup_latitude|avg_dropoff_longitude|avg_dropoff_latitude|avg_passenger_count|
+----------------+------------------+------------------+--------------------+-------------------+---------------------+--------------------+-------------------+
|8.92648807986677|               8.0| 4.139180596976789|  -72.53942610644764| 39.942037382935425|   -72.53451246878492|   39.94075474141564| 1.6849062927534162|
+----------------+------------------+------------------+--------------------+-------------------+---------------------+--------------------+-------------------+



In [26]:
# which time of the day is the busiest
busiest_hour = """
SELECT
    hour_of_day,
    COUNT(*) AS trip_count
FROM NYC_taxi_data
GROUP BY hour_of_day
ORDER BY trip_count DESC
"""
busiest_hour = spark.sql(busiest_hour)
busiest_hour.show()

+-----------+----------+
|hour_of_day|trip_count|
+-----------+----------+
|         19|     11557|
|         18|     11220|
|         20|     10871|
|         21|     10551|
|         22|     10253|
|         14|      9121|
|         12|      9023|
|         23|      8993|
|         17|      8907|
|         13|      8841|
|          9|      8714|
|         11|      8577|
|         15|      8523|
|          8|      8306|
|         10|      8203|
|         16|      7238|
|          0|      7074|
|          7|      6748|
|          1|      5150|
|          2|      3990|
+-----------+----------+
only showing top 20 rows



In [27]:
# which day of the week is the busiest day
busiest_day_of_week = """
SELECT
    day_of_week,
    COUNT(*) AS trip_count
FROM NYC_taxi_data
GROUP BY day_of_week
ORDER BY trip_count DESC
"""
busiest_day_of_week = spark.sql(busiest_day_of_week)
busiest_day_of_week.show()

+-----------+----------+
|day_of_week|trip_count|
+-----------+----------+
|          5|     28209|
|          6|     27989|
|          4|     26967|
|          3|     26543|
|          2|     25445|
|          7|     23534|
|          1|     23316|
+-----------+----------+



In [28]:
# daily trend: average fare amount and trip count per day
daily_trends = spark.sql("""
    SELECT year, month, day,
           AVG(fare_amount) AS average_fare,
           COUNT(*) AS trip_count
    FROM NYC_taxi_data
    GROUP BY year, month, day
    ORDER BY year, month, day
""")
daily_trends.show()


+----+-----+---+-----------------+----------+
|year|month|day|     average_fare|trip_count|
+----+-----+---+-----------------+----------+
|2009|    1|  1|8.135714285714284|        70|
|2009|    1|  2|7.201492537313433|        67|
|2009|    1|  3| 8.25294117647059|        68|
|2009|    1|  4|8.080952380952379|        63|
|2009|    1|  5|8.450877192982455|        57|
|2009|    1|  6|7.482278481012658|        79|
|2009|    1|  7| 7.56830985915493|        71|
|2009|    1|  8| 7.55142857142857|        70|
|2009|    1|  9|7.985294117647059|       102|
|2009|    1| 10|7.771212121212121|        66|
|2009|    1| 11| 8.15223880597015|        67|
|2009|    1| 12|8.119696969696971|        66|
|2009|    1| 13| 8.37784810126582|        79|
|2009|    1| 14|7.913333333333334|        90|
|2009|    1| 15|8.394512195121951|        82|
|2009|    1| 16|8.294565217391305|        92|
|2009|    1| 17|8.048192771084334|        83|
|2009|    1| 18|7.893442622950818|        61|
|2009|    1| 19|7.435135135135136|

In [29]:
# weekly average fare and trip count by day of the week
weekly_trends = spark.sql("""
    SELECT day_of_week,
           AVG(fare_amount) AS average_fare,
           COUNT(*) AS trip_count
    FROM NYC_taxi_data
    GROUP BY day_of_week
    ORDER BY day_of_week
""")
weekly_trends.show()


+-----------+-----------------+----------+
|day_of_week|     average_fare|trip_count|
+-----------+-----------------+----------+
|          1|8.674797135014563|     23316|
|          2|8.838727058361174|     25445|
|          3| 8.92565911916514|     26543|
|          4|8.996404494382034|     26967|
|          5|9.029982984154035|     28209|
|          6|9.018290399799902|     27989|
|          7| 8.95831987762387|     23534|
+-----------+-----------------+----------+



#Feature Engineering and Feature Selection

In [30]:
df_1=df_1.drop("pickup_datetime")

In [31]:
# select features and target variable
feature_columns = ["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude",
                   "passenger_count", "month", "day", "day_of_week", "hour_of_day"]
target_column = "fare_amount"

In [32]:
# create feature vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(df_1)

# Split Data

In [33]:
final_data = data.select("features", "fare_amount")
final_data.show(10)

+--------------------+-----------+
|            features|fare_amount|
+--------------------+-----------+
|[-73.996097,40.73...|       12.9|
|[-73.976059,40.75...|        4.5|
|[-73.994757,40.71...|       10.5|
|[-74.003652,40.74...|        6.5|
|[-73.947739,40.77...|        9.7|
|[-73.982473,40.77...|        9.3|
|[-73.99449,40.722...|        5.7|
|[-73.990755,40.77...|       11.3|
|[-73.980368,40.78...|        5.0|
|[-74.001675,40.72...|       10.0|
+--------------------+-----------+
only showing top 10 rows



In [34]:
# split the data into training and testing sets (70% training, 30% testing)
train_data, test_data = df_1.randomSplit([0.7, 0.3], seed=42)

In [35]:
train_data.show(10)

+-----------+----------------+---------------+-----------------+----------------+---------------+----+-----+---+-----------+-----------+
|fare_amount|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|year|month|day|day_of_week|hour_of_day|
+-----------+----------------+---------------+-----------------+----------------+---------------+----+-----+---+-----------+-----------+
|        0.0|      -73.942346|      40.806269|       -73.942463|       40.806129|              1|2010|    3| 18|          4|         19|
|        0.0|       -73.48448|      40.813401|       -73.484442|       40.813323|              2|2014|    6| 29|          7|         16|
|        2.5|      -74.030525|      40.748424|       -74.030494|       40.748417|              1|2012|   12| 17|          1|         22|
|        2.5|      -74.016967|      40.709565|       -74.017062|       40.709092|              1|2009|    1|  2|          5|         14|
|        2.5|      -74.014547|      40.71

# Model Selection and Training

In [36]:
# initialize the Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="fare_amount")

In [37]:
# define the parameter grid for Linear Regression
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

In [38]:
# Evaluate model
lrevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="fare_amount", metricName="rmse")

# Create 3-fold CrossValidator
lrcv = CrossValidator(estimator = lr,
                    estimatorParamMaps = paramGrid,
                    evaluator = lrevaluator,
                    numFolds = 3)

In [39]:
# pipelineing
pipeline=Pipeline(stages=[assembler,lrcv])

In [40]:
# fit the model
pipeline_model = pipeline.fit(train_data)

# Model Evaluation

In [41]:
# get predictions from the best model
predictions = pipeline_model.transform(test_data)

# evaluate the best model using RMSE
rmse = lrevaluator.evaluate(predictions)
print(f"Best Model's RMSE on test data: {rmse}")

Best Model's RMSE on test data: 4.10924319457988


In [42]:
best_model = pipeline_model.stages[-1].bestModel
best_param=best_model.extractParamMap()

for param, value in best_param.items():
    print(f"{param.name}: {value}")


aggregationDepth: 2
elasticNetParam: 0.0
epsilon: 1.35
featuresCol: features
fitIntercept: True
labelCol: fare_amount
loss: squaredError
maxBlockSizeInMB: 0.0
maxIter: 100
predictionCol: prediction
regParam: 0.01
solver: auto
standardization: True
tol: 1e-06


# Saving the model

In [45]:
pipeline_model.save('/content/drive/MyDrive/NYC_taxi_data/lr_models')